This notebook is provided as basic demonstration of use of the pmapUtilities R package, developed by Luke C. Mullany, PhD MS MHS, and available at https://github.com/lmullany/pmapUtilities.git and can be installed using `devtools::install_github("lmullany/pmapUtilities")`

- **Platform Tool** : Jupyter/RStudio Crunchr Compute Containers
- **Programming Language**: R (>=3.6)
- **Author(s)** : Luke C. Mullany
- **License** : The notebook is release under the [Apache 2.0 License] (https://www.apache.org/licenses/LICENSE-2.0)
- **Last Updated** : March 22, 2021

## Basic use of pmapUtilities R package

#### Clear workspace and load libraries

In [15]:
rm(list=ls())
library(pmapUtilities)

#### Generate a connection to the database

In [17]:
de = get_sql_connection("CAMP_PMCoE_Projection",username = "lmullan1")

Enter Password for lmullan1:  ·········


Note: name/rename your connection as 'default_engine' to avoid
 specifying an engine in subsequent pmap.utilities:: functions


#### Show all the tables in this database

In [18]:
print(list_tables(engine=de))

                 table
 1:           patients
 2:         encounters
 3:               labs
 4:               meds
 5:        problemlist
 6:         procedures
 7:           symptoms
 8:          vitals_BP
 9:      vitals_height
10:       vitals_pulse
11: vitals_respiration
12: vitals_temperature
13:      vitals_weight
14:        sysdiagrams


#### Call the same function, but this time ask for dimensions (`show_dimensions = TRUE`)

In [19]:
print(list_tables(engine=de,show_dimensions = TRUE, exact=T))

                 table    rows cols
 1:           patients   60676    5
 2:         encounters  753484    4
 3:               labs 3509868   12
 4:               meds  631022   10
 5:        problemlist  781379    4
 6:         procedures 5550672    6
 7:           symptoms 1967436    5
 8:          vitals_BP 1061684    7
 9:      vitals_height  305612    7
10:       vitals_pulse 1227099    7
11: vitals_respiration  957677    7
12: vitals_temperature  805345    7
13:      vitals_weight  406052    7
14:        sysdiagrams       0    5


#### List columns for a particular table

In [20]:
print(list_columns("encounters",engine=de))

[1] "osler_id"       "encounter_id"   "encounter_type" "encounter_date"


#### Get the number of rows and columns for just one table (rather than all tables, see above)

In [21]:
dims = get_table_dim("encounters", engine=de, exact = T)
print(dims)

  rows   cols 
753484      4 


#### Return a lazy handle to the table
Note that this feature is using dplyr/dbplyr under the hood, which enables us to translate dplyr verbs into SQL and execute on the table without pulling all rows from the table

In [22]:
encounters = return_table("encounters", engine=de)
encounters

# Source:   SQL [?? x 4]
# Database: Microsoft SQL Server 13.00.5830[@ESMPMDBPR4/CAMP_PMCoE_Projection]
   osler_id                      encounter_id encounter_type encounter_date     
   <chr>                                <int> <chr>          <dttm>             
 1 5303550b-8ed2-42fd-885a-d32b~        17938 Office Visit   2015-10-10 00:00:00
 2 5303550b-8ed2-42fd-885a-d32b~       142706 Office Visit   2016-01-24 00:00:00
 3 5303550b-8ed2-42fd-885a-d32b~       571465 Office Visit   2017-03-19 00:00:00
 4 5303550b-8ed2-42fd-885a-d32b~       432470 Office Visit   2016-10-22 00:00:00
 5 5303550b-8ed2-42fd-885a-d32b~       410795 Office Visit   2016-10-01 00:00:00
 6 5303550b-8ed2-42fd-885a-d32b~       631974 Office Visit   2017-05-08 00:00:00
 7 5303550b-8ed2-42fd-885a-d32b~       483944 Office Visit   2016-12-10 00:00:00
 8 5303550b-8ed2-42fd-885a-d32b~       411794 Office Visit   2016-10-03 00:00:00
 9 5303550b-8ed2-42fd-885a-d32b~       324374 Office Visit   2016-07-11 00:00:00
10 53

For example, we can do "SELECT encounter_type, count() as CT from encounters group by encounter_type ORDER BY COUNT()" without explicitly writing the SQL query

In [23]:
encounters %>% dplyr::group_by(encounter_type) %>% dplyr::summarize(ct = n()) %>% dplyr::arrange(desc(ct))

# Source:     lazy query [?? x 2]
# Database:   Microsoft SQL Server
#   13.00.5830[@ESMPMDBPR4/CAMP_PMCoE_Projection]
# Ordered by: desc(ct)
   encounter_type         ct
   <chr>               <int>
 1 Office Visit       431080
 2 Appointment         97718
 3 Hospital Encounter  86789
 4 Visit Encounter     62740
 5 Clinical Support    21626
 6 Procedure visit     18256
 7 Results Only         8895
 8 Orders Only          8635
 9 Anti-coag visit      7284
10 Provider Procedure   2987
# ... with more rows

We can of course pull the entire table locally, if we desire, using `dplyr::collect()`

In [24]:
encounters_local = encounters %>% dplyr::collect()
print(encounters_local %>% head(10))

# A tibble: 10 x 4
   osler_id                      encounter_id encounter_type encounter_date     
   <chr>                                <int> <chr>          <dttm>             
 1 5303550b-8ed2-42fd-885a-d32b~        17938 Office Visit   2015-10-10 00:00:00
 2 5303550b-8ed2-42fd-885a-d32b~       142706 Office Visit   2016-01-24 00:00:00
 3 5303550b-8ed2-42fd-885a-d32b~       571465 Office Visit   2017-03-19 00:00:00
 4 5303550b-8ed2-42fd-885a-d32b~       432470 Office Visit   2016-10-22 00:00:00
 5 5303550b-8ed2-42fd-885a-d32b~       410795 Office Visit   2016-10-01 00:00:00
 6 5303550b-8ed2-42fd-885a-d32b~       631974 Office Visit   2017-05-08 00:00:00
 7 5303550b-8ed2-42fd-885a-d32b~       483944 Office Visit   2016-12-10 00:00:00
 8 5303550b-8ed2-42fd-885a-d32b~       411794 Office Visit   2016-10-03 00:00:00
 9 5303550b-8ed2-42fd-885a-d32b~       324374 Office Visit   2016-07-11 00:00:00
10 5303550b-8ed2-42fd-885a-d32b~       705948 Office Visit   2017-07-19 00:00:00


#### Use `query_db()` to submit any sql query directly to the database; by default it returns a lazy tbl

In [25]:
qry <- "SELECT encounter_type, ct = COUNT(*) FROM encounters GROUP BY encounter_type"
query_db(qry, engine=de) %>% dplyr::arrange(desc(ct))

# Source:     SQL [?? x 2]
# Database:   Microsoft SQL Server
#   13.00.5830[@ESMPMDBPR4/CAMP_PMCoE_Projection]
# Ordered by: desc(ct)
   encounter_type         ct
   <chr>               <int>
 1 Office Visit       431080
 2 Appointment         97718
 3 Hospital Encounter  86789
 4 Visit Encounter     62740
 5 Clinical Support    21626
 6 Procedure visit     18256
 7 Results Only         8895
 8 Orders Only          8635
 9 Anti-coag visit      7284
10 Provider Procedure   2987
# ... with more rows

#### Use gen_random_table() to get subset of a table

In [34]:
# generate the temp table; the function returns the name of the temp table
random_px = gen_random_table("patients",idvars="osler_id", engine=de)
cat("Name of the newly created table is ", random_px,"\n")

Name of the newly created table is  #31e5ixaf9t0d8yr9 


In [39]:
return_table(random_px, engine=de) %>%
    dplyr::count()


# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.5830[@ESMPMDBPR4/CAMP_PMCoE_Projection]
      n
  <int>
1  1000